# get data

In [1]:
import sys
sys.path.append('scripts')
from data_generator import *

In [17]:
def data_process(SUBJ, 
                 res='/Volumes/BPlus/primeRec/results/',
                 data=data='/Volumes/BPlus/primeRec/data/'):
     
    dg = DataGenerator(SUBJ, res, data)
    info_list = dg.get_info_list()
    lat_arr = dg.get_latent(info_list)
    voxel_arr = dg.get_voxel(info_list)
    voxel_masked=dg.get_masked_voxel(voxel_arr)
    print(lat_arr.shape, voxel_arr.shape, voxel_masked.shape)

    save_path=res+SUBJ+'/regression'
    np.save('%s/lat_arr'%save_path, lat_arr)
    np.save('%s/voxel_arr'%save_path, voxel_arr)
    np.save('%s/voxel_masked'%save_path, voxel_masked)

In [38]:
data_process('sub02')

* number of infos: 1000
* smaple info:
[[1, 1], '/Volumes/BPlus/primeRec/results//sub02/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat1_mask1.nii.gz', [1, 583]]
(1000, 100) (1000, 91, 109, 91) (1000, 17540)


In [39]:
data_process('sub19')

* number of infos: 1000
* smaple info:
[[1, 1], '/Volumes/BPlus/primeRec/results//sub19/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat1_mask1.nii.gz', [2, 219]]
(1000, 100) (1000, 91, 109, 91) (1000, 24452)


In [18]:
data_process('sub21')

* number of infos: 1000
* smaple info:
[[1, 1], '/Volumes/BPlus/primeRec/results//sub21/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat1_mask1.nii.gz', [1, 23]]
(1000, 100) (1000, 91, 109, 91) (1000, 20198)


In [19]:
data_process('sub22')

* number of infos: 1000
* smaple info:
[[1, 1], '/Volumes/BPlus/primeRec/results//sub22/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat1_mask1.nii.gz', [1, 453]]
(1000, 100) (1000, 91, 109, 91) (1000, 27347)


In [20]:
data_process('sub23')

* number of infos: 1000
* smaple info:
[[1, 1], '/Volumes/BPlus/primeRec/results//sub23/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat1_mask1.nii.gz', [1, 509]]
(1000, 100) (1000, 91, 109, 91) (1000, 21395)


In [21]:
data_process('sub24')

* number of infos: 1000
* smaple info:
[[1, 2], '/Volumes/BPlus/primeRec/results//sub24/trial_wise/phase1_trialWiseGLM_train_run1.feat/stats/tstat2_mask1.nii.gz', [1, 342]]
(1000, 100) (1000, 91, 109, 91) (1000, 23717)


# Start regression and get Corr

In [8]:
from regression_yb import *

In [32]:
def regression_process(SUBJ, res='/Volumes/BPlus/primeRec/results/'):

    load_path=res+SUBJ+'/regression'
    lat_arr = np.load('%s/lat_arr.npy'%load_path)
    voxel_masked = np.load('%s/voxel_masked.npy'%load_path)
    print(lat_arr.shape, voxel_masked.shape)

    corr, self, others = regression_get_corr(voxel_masked, lat_arr)
    print('* Mean: ', np.mean(self), np.mean(others))

In [40]:
regression_process('sub02')

(1000, 100) (1000, 17540)
0.1871607667985445 0.15205739380074326
0.2056761857135844 0.15958325132077264
0.2309484086064575 0.18081680360512906
0.1800697743812806 0.15835770327686624
0.15989728061125835 0.14561444094727666
0.21344214069783515 0.1903536759692428
0.21375614103837162 0.17920020654197297
0.18341719044107038 0.14898396291288252
0.19395665235252896 0.1652407484306911
0.20381520437588616 0.15986098726365328
* Mean:  0.19721397450168174 0.16400691740692305


In [41]:
regression_process('sub19')

(1000, 100) (1000, 24452)
0.2451471354582144 0.18473068786732647
0.22291207843188066 0.18295813496691143
0.20391077584634387 0.1740821463219225
0.20479170706474373 0.16440069125794074
0.21654354693929728 0.18767342027460804
0.21199406251401623 0.1822509165158312
0.21497325506692935 0.1853725366278745
0.17228619531987946 0.1615730647795519
0.20934973799460885 0.19397601610477663
0.2413833937474849 0.1870295891430919
* Mean:  0.21432918883833985 0.1804047203859835


In [33]:
regression_process('sub21')

(1000, 100) (1000, 20198)
0.24677335013995158 0.20243538019141624
0.1904589239511496 0.15934156567803595
0.2082733134853766 0.19300538878957013
0.20680193800428587 0.17021743244514548
0.20822471535115508 0.17374615527309195
0.22352110670350864 0.18097707625662765
0.16840334415023472 0.15260818982681446
0.20463280520818233 0.183505679423554
0.20161499930740662 0.16572306729666275
0.22145315730176737 0.2024996864050678
* Mean:  0.20801576536030186 0.17840596215859866


In [34]:
regression_process('sub22')

(1000, 100) (1000, 27347)
0.19889642970291177 0.17658899321074392
0.23064028271267464 0.18663994863283337
0.2026919044457852 0.191080074006556
0.22530791200022318 0.19994010290252953
0.17450955886126768 0.1757699771301019
0.20041546867664947 0.18866169505314984
0.22975023025473948 0.20719153105035143
0.2021562232632305 0.19613038578895034
0.19524705335889553 0.17035312214062992
0.20802686431357284 0.19968256546870275
* Mean:  0.20676419275899502 0.1892038395384549


In [35]:
regression_process('sub23')

(1000, 100) (1000, 21395)
0.25032566591032734 0.17250031128545043
0.24074908744167267 0.1779538414794357
0.2318232113787796 0.1692300314597998
0.26928879533423555 0.1978144924222298
0.24938165572063084 0.1850253424459345
0.23160803044879733 0.17710322624048966
0.26029916989614393 0.16794782571261768
0.21345859652239663 0.156805057258255
0.24523123347340386 0.18409494237077084
0.243130506583305 0.1705231876016818
* Mean:  0.24352959527096923 0.17589982582766653


In [36]:
regression_process('sub24')

(1000, 100) (1000, 23717)
0.2020010553094862 0.18789764184404664
0.19690240975898923 0.17978640430506454
0.20487443759447344 0.17020147896400437
0.20497596061029824 0.16985753919357402
0.20448269587061993 0.18182201622728594
0.1850276302164123 0.16252846980892044
0.20192824136937426 0.1905257387903996
0.21901068861373857 0.18661472987437266
0.20288936589876438 0.17215733570368724
0.22811834589325997 0.19330447175327972
* Mean:  0.20502108311354167 0.1794695826464635
